In [307]:
#Cargamos las librerías necesarias 
import pandas as pd
import pymongo 
import holidays
from prophet import Prophet
import matplotlib.pyplot as plt
import numpy as np

En primer lugar descargamos los datos para los predictores del SCM, de la siguiente base de datos. Contiene indicadores urbanos para distintas temáticas como el mercado laboral, variables demográficas, educaciñon, transporte, condiciones de vida... https://ec.europa.eu/eurostat/web/cities/data/database

In [308]:
#Conectamos a MongoDB para obtener los datos que requerimos para nuestra investigación

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client['Covid_19_Caro_Gutierrez'] 
collection = db['urban_indicators']
# Consulta MongoDB y carga los documentos en un cursor
data = collection.find()


In [309]:
# Convierte los datos en un DataFrame de Pandas
df = pd.DataFrame(list(data))
df.drop('_id', axis=1, inplace=True)


In [310]:
# Separar la columna en dos columnas usando la coma como separador
split_columns = df['indic_ur,cities\\time'].str.split(',', expand=True)

# Renombrar las nuevas columnas
split_columns.columns = ['indicator', 'city']

# Reemplazar las columnas originales con las nuevas columnas en su misma posición
df = pd.concat([df, split_columns], axis=1)
df = df.drop(columns=['indic_ur,cities\\time'])

columnas_indicadores_ciudades = ['indicator', 'city']

# Reorganizar las columnas para que los indicadores y las ciudades estén a la izquierda
nuevas_columnas = columnas_indicadores_ciudades + [col for col in df.columns if col not in columnas_indicadores_ciudades]

# Crear un nuevo DataFrame con las columnas reorganizadas
df = df[nuevas_columnas]
df

,indicator,city,2022,2021,2020,2019,2018,2017,2016,2015,...,1998,1997,1996,1995,1994,1993,1992,1991,1990,1989
0,EC2021V,AT,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,NaN,NaN
1,EC2021V,AT001C,:,:,:,:,:,:,:,:,...,:,:,87071,:,:,:,:,:,NaN,NaN
2,EC2021V,AT002C,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,NaN,NaN
3,EC2021V,AT003C,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,NaN,NaN
4,EC2021V,AT004C,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147489,TT1081V,UK025C,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
147490,TT1081V,UK026C,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
147491,TT1081V,UK027C,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:
147492,TT1081V,UK028C,:,:,:,:,:,:,:,:,...,:,:,:,:,:,:,:,:,:,:


In [311]:
#Conertimos en formato amigable para SCM
# Lista de columnas a mantener fijas (en este caso, 'Indicator' y 'City')
id_vars = ['indicator', 'city']

# Utiliza pd.melt() para derretir las columnas de años
df = pd.melt(df, id_vars=id_vars, var_name='year', value_name='value')
df

,indicator,city,year,value
0,EC2021V,AT,2022,:
1,EC2021V,AT001C,2022,:
2,EC2021V,AT002C,2022,:
3,EC2021V,AT003C,2022,:
4,EC2021V,AT004C,2022,:
...,...,...,...,...
5014791,TT1081V,UK025C,1989,:
5014792,TT1081V,UK026C,1989,:
5014793,TT1081V,UK027C,1989,:
5014794,TT1081V,UK028C,1989,:


In [312]:
# Crea una tabla pivote con los indicadores como columnas
df = df.pivot(index=['city', 'year'], columns='indicator', values='value')

In [313]:
# Lista de columnas involucradas en las operaciones
columns_to_clean = ['EC2008V', 'EC2020V', 'EC2009V', 'EC2022V', 'TE2028V', 'DE1058V', 'DE1061V', 'DE1064V',
                    'TE2031V', 'DE1040V', 'DE1074V', 'DE1077V', 'DE1046V', 'DE1028V', 'DE1055V', 'DE1049V',
                    'DE1058V', 'DE1061V', 'DE1064V', 'DE1025V', 'DE1003V', 'DE1002V', 'EC1020I', 'EC1011V',
                    'EC1002V', 'EC1012V', 'EC1003V', 'DE2003I']

for column in columns_to_clean:
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [314]:
# Reemplazar celdas vacías con NaN
df = df.replace('', float('nan'))

In [315]:

# Share of industry by city
df['agriculture'] = df['EC2008V'] / df['EC2020V']
df['industry'] = df['EC2009V'] / df['EC2020V']
df['construction'] = df['EC2022V'] / df['EC2020V']  # Corregido el nombre de la columna

# Share of education attainment
df['edu2'] = df['TE2028V'] / (df['DE1058V'] + df['DE1061V'] + df['DE1064V'])
df['edu3'] = df['TE2031V'] / (df['DE1058V'] + df['DE1061V'] + df['DE1064V'])

# Age-dependency ratios
df['age_dependency'] = (df['DE1040V'] + df['DE1074V'] + df['DE1077V'] + df['DE1046V'] + df['DE1028V'] + df['DE1055V']) / (df['DE1049V'] + df['DE1058V'] + df['DE1061V'] + df['DE1064V'] + df['DE1025V'])
df['old_age_dependency'] = (df['DE1040V'] + df['DE1074V'] + df['DE1077V'] + df['DE1046V']) / (df['DE1049V'] + df['DE1058V'] + df['DE1061V'] + df['DE1064V'] + df['DE1025V'])
df['young_age_dependency'] = (df['DE1028V'] + df['DE1055V']) / (df['DE1049V'] + df['DE1058V'] + df['DE1061V'] + df['DE1064V'] + df['DE1025V'])

# Sex ratio
df['sex_ratio'] = df['DE1003V'] * 100 / df['DE1002V']

# Unemployment
df['unemp'] = df['EC1020I']
df['f_unemp'] = df['EC1011V'] / df['EC1002V']
df['m_unemp'] = df['EC1012V'] / df['EC1003V']

# Inmigración
df['foreigners'] = df['DE2003I']




In [316]:
df.columns

Index(['DE1001V', 'DE1002V', 'DE1003V', 'DE1025V', 'DE1026V', 'DE1027V',
       'DE1028V', 'DE1029V', 'DE1030V', 'DE1040V',
       ...
       'edu2', 'edu3', 'age_dependency', 'old_age_dependency',
       'young_age_dependency', 'sex_ratio', 'unemp', 'f_unemp', 'm_unemp',
       'foreigners'],
      dtype='object', name='indicator', length=242)

In [317]:
# Resetear el índice y renombrar las columnas
df.reset_index(inplace=True)
df.rename(columns={'level_0': 'city', 'level_1': 'year'}, inplace=True)


In [318]:
columns_involved = ['agriculture', 'industry', 'construction', 'edu2', 'edu3', 'age_dependency', 'old_age_dependency', 'young_age_dependency', 'sex_ratio', 'unemp', 'f_unemp', 'm_unemp', 'foreigners']

df = df[['city', 'year'] + columns_involved].reset_index()


In [319]:
df

indicator,index,city,year,agriculture,industry,construction,edu2,edu3,age_dependency,old_age_dependency,young_age_dependency,sex_ratio,unemp,f_unemp,m_unemp,foreigners
0,0,AT,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AT,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AT,1991,0.174790,0.209227,0.068113,NaN,NaN,NaN,NaN,NaN,107.914566,NaN,NaN,NaN,6.0
3,3,AT,1992,0.169699,0.203311,0.070382,NaN,NaN,NaN,NaN,NaN,107.306779,NaN,NaN,NaN,NaN
4,4,AT,1993,0.166349,0.196566,0.070914,NaN,NaN,NaN,NaN,NaN,106.821174,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30935,30935,UK135C,2018,NaN,NaN,NaN,NaN,NaN,0.721748,0.468193,0.253556,103.581182,NaN,NaN,NaN,NaN
30936,30936,UK135C,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30937,30937,UK135C,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30938,30938,UK135C,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [320]:
#Asignamos el ultimo dato disponible a cada columna para tener siempre valores 
#df[columns_involved] = df[columns_involved].fillna(method='ffill')

In [321]:
# Cargar el archivo XLSX con la correspondencia entre códigos y nombres de ciudades
ruta_archivo = r'C:\Users\34645\Desktop\Doctorado_II\GoogleTrends_Caro\Datos\GTData\urb_esms_an4.xlsx'
df_hoja2 = pd.read_excel(ruta_archivo, sheet_name=2)

df = pd.merge(df, df_hoja2, how='left', left_on='city', right_on='CODE')

# Elimina la columna 'City' original si lo deseas
#df = df.drop(columns='city')


In [322]:
df

,index,city,year,agriculture,industry,construction,edu2,edu3,age_dependency,old_age_dependency,...,sex_ratio,unemp,f_unemp,m_unemp,foreigners,CODE,NAME,Ciudad,State/Province,iso_code
0,0,AT,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,AT,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,AT,1991,0.174790,0.209227,0.068113,NaN,NaN,NaN,NaN,...,107.914566,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN
3,3,AT,1992,0.169699,0.203311,0.070382,NaN,NaN,NaN,NaN,...,107.306779,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,AT,1993,0.166349,0.196566,0.070914,NaN,NaN,NaN,NaN,...,106.821174,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30935,30935,UK135C,2018,NaN,NaN,NaN,NaN,NaN,0.721748,0.468193,...,103.581182,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30936,30936,UK135C,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30937,30937,UK135C,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30938,30938,UK135C,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [323]:
df = df.dropna(subset=['iso_code'])

df

,index,city,year,agriculture,industry,construction,edu2,edu3,age_dependency,old_age_dependency,...,sex_ratio,unemp,f_unemp,m_unemp,foreigners,CODE,NAME,Ciudad,State/Province,iso_code
2550,2550,DE001C,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DE001C,Berlin,Berlin,Berlín,DE-BE
2551,2551,DE001C,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DE001C,Berlin,Berlin,Berlín,DE-BE
2552,2552,DE001C,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.0,0.076768,0.074879,NaN,DE001C,Berlin,Berlin,Berlín,DE-BE
2553,2553,DE001C,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,109.224858,NaN,NaN,NaN,10.0,DE001C,Berlin,Berlin,Berlín,DE-BE
2554,2554,DE001C,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DE001C,Berlin,Berlin,Berlín,DE-BE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24985,24985,SE014C,2018,0.007359,0.098884,0.06905,0.605625,0.396764,0.7469,0.411417,...,100.948399,7.0,0.070712,0.073823,6.0,SE014C,Borås,NaN,Borås,SE-P
24986,24986,SE014C,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24987,24987,SE014C,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24988,24988,SE014C,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P


In [324]:
paris = df[df['city']=='SE014C']
paris

,index,city,year,agriculture,industry,construction,edu2,edu3,age_dependency,old_age_dependency,...,sex_ratio,unemp,f_unemp,m_unemp,foreigners,CODE,NAME,Ciudad,State/Province,iso_code
24956,24956,SE014C,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24957,24957,SE014C,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24958,24958,SE014C,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24959,24959,SE014C,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24960,24960,SE014C,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24961,24961,SE014C,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24962,24962,SE014C,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24963,24963,SE014C,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24964,24964,SE014C,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P
24965,24965,SE014C,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,SE014C,Borås,NaN,Borås,SE-P


In [325]:
#Rellenamos aquellos datos con la última observaciones posibles

ciudades = df['city'].unique()

for ciudad in ciudades:
    mask = df['city'] == ciudad
    df.loc[mask] = df.loc[mask].fillna(method='ffill')

C:\Users\34645\AppData\Local\Temp\ipykernel_14300\1085011973.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.loc[mask] = df.loc[mask].fillna(method='ffill')


In [326]:
paris = df[df['city']=='DE001C']
paris

,index,city,year,agriculture,industry,construction,edu2,edu3,age_dependency,old_age_dependency,...,sex_ratio,unemp,f_unemp,m_unemp,foreigners,CODE,NAME,Ciudad,State/Province,iso_code
2550,2550,DE001C,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DE001C,Berlin,Berlin,Berlín,DE-BE
2551,2551,DE001C,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,DE001C,Berlin,Berlin,Berlín,DE-BE
2552,2552,DE001C,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.0,0.076768,0.074879,NaN,DE001C,Berlin,Berlin,Berlín,DE-BE
2553,2553,DE001C,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,109.224858,8.0,0.076768,0.074879,10.0,DE001C,Berlin,Berlin,Berlín,DE-BE
2554,2554,DE001C,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,109.224858,8.0,0.076768,0.074879,10.0,DE001C,Berlin,Berlin,Berlín,DE-BE
2555,2555,DE001C,1994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,109.224858,8.0,0.076768,0.074879,10.0,DE001C,Berlin,Berlin,Berlín,DE-BE
2556,2556,DE001C,1995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,109.224858,8.0,0.076768,0.074879,10.0,DE001C,Berlin,Berlin,Berlín,DE-BE
2557,2557,DE001C,1996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.733172,13.0,0.135385,0.126367,11.0,DE001C,Berlin,Berlin,Berlín,DE-BE
2558,2558,DE001C,1997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.733172,13.0,0.135385,0.126367,11.0,DE001C,Berlin,Berlin,Berlín,DE-BE
2559,2559,DE001C,1998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,106.733172,13.0,0.135385,0.126367,11.0,DE001C,Berlin,Berlin,Berlín,DE-BE


In [327]:
#Eliminar columnas no deseadas

df = df.drop(['index','city', 'Ciudad', 'State/Province'], axis=1)


In [328]:
df

,year,agriculture,industry,construction,edu2,edu3,age_dependency,old_age_dependency,young_age_dependency,sex_ratio,unemp,f_unemp,m_unemp,foreigners,CODE,NAME,iso_code
2550,1989,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE001C,Berlin,DE-BE
2551,1990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DE001C,Berlin,DE-BE
2552,1991,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,0.076768,0.074879,NaN,DE001C,Berlin,DE-BE
2553,1992,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.224858,8.0,0.076768,0.074879,10.0,DE001C,Berlin,DE-BE
2554,1993,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109.224858,8.0,0.076768,0.074879,10.0,DE001C,Berlin,DE-BE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24985,2018,0.007359,0.098884,0.06905,0.605625,0.396764,0.7469,0.411417,0.335484,100.948399,7.0,0.070712,0.073823,6.0,SE014C,Borås,SE-P
24986,2019,0.007359,0.098884,0.06905,0.605625,0.396764,0.7469,0.411417,0.335484,100.948399,7.0,0.070712,0.073823,6.0,SE014C,Borås,SE-P
24987,2020,0.007359,0.098884,0.06905,0.605625,0.396764,0.7469,0.411417,0.335484,100.948399,7.0,0.070712,0.073823,6.0,SE014C,Borås,SE-P
24988,2021,0.007359,0.098884,0.06905,0.605625,0.396764,0.7469,0.411417,0.335484,100.948399,7.0,0.070712,0.073823,6.0,SE014C,Borås,SE-P


In [329]:
# Definir el orden deseado de las columnas
nuevo_orden = ['CODE',	'NAME',	'iso_code', 'year',	'agriculture',	'industry'	,'construction',	'edu2'	,'edu3',	'age_dependency',	'old_age_dependency',	'young_age_dependency',	
               'sex_ratio',	'unemp',	'f_unemp',	'm_unemp',	'foreigners']

# Reorganizar las columnas
df = df[nuevo_orden]

In [330]:
print(df['year'].dtype)

object


In [331]:
# Limpia los valores no válidos (por ejemplo, espacios en blanco) de la columna "year"
df['year'] = df['year'].str.strip()

# Filtra los valores no vacíos antes de la conversión
df = df[df['year'].str.isnumeric()]

# Convierte la columna "year" en un objeto de fecha con el formato "%Y" para el año
df['year'] = pd.to_datetime(df['year'], format='%Y')

In [332]:
# Crea una fecha de referencia para el año 2019
fecha_referencia = pd.to_datetime('2019-01-01')

# Filtra las filas con años mayores a 2019
df = df[df['year'] > fecha_referencia]

df['year'] = pd.to_datetime(df['year'], format='%Y')
df['year'] = df['year'].dt.year

df

,CODE,NAME,iso_code,year,agriculture,industry,construction,edu2,edu3,age_dependency,old_age_dependency,young_age_dependency,sex_ratio,unemp,f_unemp,m_unemp,foreigners
2581,DE001C,Berlin,DE-BE,2020,0.000419,0.076745,0.046983,NaN,NaN,0.592713,0.290913,0.306017,103.377815,8.0,0.086068,0.071170,9.0
2582,DE001C,Berlin,DE-BE,2021,0.000419,0.076745,0.046983,NaN,NaN,0.592713,0.290913,0.306017,103.330229,8.0,0.086068,0.071170,9.0
2583,DE001C,Berlin,DE-BE,2022,0.000419,0.076745,0.046983,NaN,NaN,0.592713,0.290913,0.306017,103.330229,8.0,0.086068,0.071170,9.0
2615,DE002C,Hamburg,DE-HH,2020,0.001239,0.101861,0.035432,NaN,NaN,0.583086,0.295939,0.290849,104.348023,4.0,0.046142,0.036989,9.0
2616,DE002C,Hamburg,DE-HH,2021,0.001239,0.101861,0.035432,NaN,NaN,0.583086,0.295939,0.290849,104.257426,4.0,0.046142,0.036989,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24954,SE013C,Lund,SE-M,2021,0.008420,0.063090,0.034359,0.337775,0.764376,0.627271,0.358600,0.268671,101.311377,6.0,0.065544,0.053642,6.0
24955,SE013C,Lund,SE-M,2022,0.008420,0.063090,0.034359,0.337775,0.764376,0.627271,0.358600,0.268671,101.311377,6.0,0.065544,0.053642,6.0
24987,SE014C,Borås,SE-P,2020,0.007359,0.098884,0.069050,0.605625,0.396764,0.746900,0.411417,0.335484,100.948399,7.0,0.070712,0.073823,6.0
24988,SE014C,Borås,SE-P,2021,0.007359,0.098884,0.069050,0.605625,0.396764,0.746900,0.411417,0.335484,100.948399,7.0,0.070712,0.073823,6.0


In [333]:
# Ruta del archivo CSV que deseas cargar
ruta_archivo_csv = r'C:\Users\34645\Desktop\Doctorado_II\Códigos_Python\IEDIS\Data\eu_data.csv'

# Cargar los datos desde el archivo CSV en un nuevo DataFrame
datos_mob = pd.read_csv(ruta_archivo_csv)

In [334]:
datos_mob = datos_mob.rename(columns={'iso_3166_2_code': 'iso_code'})

In [335]:
datos_mob['date'] = pd.to_datetime(datos_mob['date'])


In [336]:
datos_mob['year'] = pd.to_datetime(datos_mob['date']).dt.year

In [337]:
datos_mob

,country,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_raw,residential_percent_change_from_baseline,sub_region_1,iso_code,...,Valor_Stringency_Index,Valor_GovernmentResponseIndex,Valor_EconomicSupportIndex,Valor_Containment_Health_index,workplace_closing,daily_cases,d2,d1,Periodo,year
0,Germany,2020-02-15,6.0,1.0,52.0,10.0,3.0,-1.0,Baden-Württemberg,DE-BW,...,0.00,0.00,0.0,0.00,0.0,16.0,0,0,1,2020
1,Germany,2020-02-16,23.0,43.0,64.0,20.0,1.0,-2.0,Baden-Württemberg,DE-BW,...,0.00,0.00,0.0,0.00,0.0,16.0,0,0,2,2020
2,Germany,2020-02-17,0.0,-1.0,-11.0,0.0,-1.0,1.0,Baden-Württemberg,DE-BW,...,0.00,0.00,0.0,0.00,0.0,16.0,0,0,3,2020
3,Germany,2020-02-18,4.0,3.0,16.0,1.0,0.0,1.0,Baden-Württemberg,DE-BW,...,0.00,0.00,0.0,0.00,0.0,16.0,0,0,4,2020
4,Germany,2020-02-19,2.0,0.0,3.0,-2.0,-1.0,0.0,Baden-Württemberg,DE-BW,...,0.00,0.00,0.0,0.00,0.0,16.0,0,0,5,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324930,Sweden,2022-10-11,-5.0,7.0,92.0,-16.0,-12.0,2.0,Västra Götaland County,SE-O,...,11.11,14.58,0.0,16.67,0.0,0.0,1,1,970,2022
324931,Sweden,2022-10-12,-7.0,4.0,59.0,-22.0,-12.0,3.0,Västra Götaland County,SE-O,...,11.11,14.58,0.0,16.67,0.0,0.0,1,1,971,2022
324932,Sweden,2022-10-13,-6.0,7.0,55.0,-19.0,-11.0,3.0,Västra Götaland County,SE-O,...,11.11,14.58,0.0,16.67,0.0,0.0,1,1,972,2022
324933,Sweden,2022-10-14,-4.0,6.0,69.0,-18.0,-14.0,3.0,Västra Götaland County,SE-O,...,11.11,14.58,0.0,16.67,0.0,0.0,1,1,973,2022


In [338]:
print(datos_mob['year'].dtype)
print(df['year'].dtype)

int32
int32


In [339]:
df

,CODE,NAME,iso_code,year,agriculture,industry,construction,edu2,edu3,age_dependency,old_age_dependency,young_age_dependency,sex_ratio,unemp,f_unemp,m_unemp,foreigners
2581,DE001C,Berlin,DE-BE,2020,0.000419,0.076745,0.046983,NaN,NaN,0.592713,0.290913,0.306017,103.377815,8.0,0.086068,0.071170,9.0
2582,DE001C,Berlin,DE-BE,2021,0.000419,0.076745,0.046983,NaN,NaN,0.592713,0.290913,0.306017,103.330229,8.0,0.086068,0.071170,9.0
2583,DE001C,Berlin,DE-BE,2022,0.000419,0.076745,0.046983,NaN,NaN,0.592713,0.290913,0.306017,103.330229,8.0,0.086068,0.071170,9.0
2615,DE002C,Hamburg,DE-HH,2020,0.001239,0.101861,0.035432,NaN,NaN,0.583086,0.295939,0.290849,104.348023,4.0,0.046142,0.036989,9.0
2616,DE002C,Hamburg,DE-HH,2021,0.001239,0.101861,0.035432,NaN,NaN,0.583086,0.295939,0.290849,104.257426,4.0,0.046142,0.036989,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24954,SE013C,Lund,SE-M,2021,0.008420,0.063090,0.034359,0.337775,0.764376,0.627271,0.358600,0.268671,101.311377,6.0,0.065544,0.053642,6.0
24955,SE013C,Lund,SE-M,2022,0.008420,0.063090,0.034359,0.337775,0.764376,0.627271,0.358600,0.268671,101.311377,6.0,0.065544,0.053642,6.0
24987,SE014C,Borås,SE-P,2020,0.007359,0.098884,0.069050,0.605625,0.396764,0.746900,0.411417,0.335484,100.948399,7.0,0.070712,0.073823,6.0
24988,SE014C,Borås,SE-P,2021,0.007359,0.098884,0.069050,0.605625,0.396764,0.746900,0.411417,0.335484,100.948399,7.0,0.070712,0.073823,6.0


In [341]:

# Realiza la combinación basada en la columna 'year_date' y 'iso_code'
dataset_daily = datos_mob.merge(df, left_on=['year', 'iso_code'], right_on=['year', 'iso_code'], how='left').fillna(method='ffill')


C:\Users\34645\AppData\Local\Temp\ipykernel_14300\1809871878.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset_daily = datos_mob.merge(df, left_on=['year', 'iso_code'], right_on=['year', 'iso_code'], how='left').fillna(method='ffill')


In [342]:
dataset_daily

,country,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_raw,residential_percent_change_from_baseline,sub_region_1,iso_code,...,edu2,edu3,age_dependency,old_age_dependency,young_age_dependency,sex_ratio,unemp,f_unemp,m_unemp,foreigners
0,Germany,2020-02-15,6.0,1.0,52.0,10.0,3.0,-1.0,Baden-Württemberg,DE-BW,...,NaN,NaN,0.543302,0.265401,0.275822,100.101638,4.0,0.046437,0.042374,11.0
1,Germany,2020-02-15,6.0,1.0,52.0,10.0,3.0,-1.0,Baden-Württemberg,DE-BW,...,NaN,NaN,0.529530,0.278593,0.249795,109.542929,5.0,0.055947,0.043759,10.0
2,Germany,2020-02-15,6.0,1.0,52.0,10.0,3.0,-1.0,Baden-Württemberg,DE-BW,...,NaN,NaN,0.537433,0.255138,0.282108,95.175343,4.0,0.042998,0.039191,10.0
3,Germany,2020-02-15,6.0,1.0,52.0,10.0,3.0,-1.0,Baden-Württemberg,DE-BW,...,NaN,NaN,0.635839,0.301712,0.329405,99.586602,4.0,0.042482,0.039306,10.0
4,Germany,2020-02-15,6.0,1.0,52.0,10.0,3.0,-1.0,Baden-Württemberg,DE-BW,...,NaN,NaN,0.616292,0.302057,0.311643,102.694390,4.0,0.037524,0.037802,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
557713,Sweden,2022-10-11,-5.0,7.0,92.0,-16.0,-12.0,2.0,Västra Götaland County,SE-O,...,0.446133,0.556488,0.588936,0.344922,0.244014,100.354148,7.0,0.074715,0.066655,7.0
557714,Sweden,2022-10-12,-7.0,4.0,59.0,-22.0,-12.0,3.0,Västra Götaland County,SE-O,...,0.446133,0.556488,0.588936,0.344922,0.244014,100.354148,7.0,0.074715,0.066655,7.0
557715,Sweden,2022-10-13,-6.0,7.0,55.0,-19.0,-11.0,3.0,Västra Götaland County,SE-O,...,0.446133,0.556488,0.588936,0.344922,0.244014,100.354148,7.0,0.074715,0.066655,7.0
557716,Sweden,2022-10-14,-4.0,6.0,69.0,-18.0,-14.0,3.0,Västra Götaland County,SE-O,...,0.446133,0.556488,0.588936,0.344922,0.244014,100.354148,7.0,0.074715,0.066655,7.0


In [343]:
ruta_archivo_csv = r'C:\Users\34645\Desktop\Doctorado_II\Códigos_Python\IEDIS\Data\eu_dailydataset.csv'
dataset_daily.to_csv(ruta_archivo_csv, index=False)  # index=False para no incluir el índice en el archivo CSV

In [344]:
# Ruta del archivo CSV que deseas cargar
ruta_archivo_csv = r'C:\Users\34645\Desktop\Doctorado_II\Códigos_Python\IEDIS\Data\eu_data_filtered_w.csv'

# Cargar los datos desde el archivo CSV en un nuevo DataFrame
datos_mob = pd.read_csv(ruta_archivo_csv)

In [345]:
datos_mob

,iso_3166_2_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_raw,residential_percent_change_from_baseline,trend,workplaces,Valor_Stringency_Index,Valor_GovernmentResponseIndex,Valor_EconomicSupportIndex,Valor_Containment_Health_index,workplace_closing,daily_cases,d2,d1,Periodo,Week
0,DE-BB,2020-02-16,6.500000,2.500000,17.000000,4.500000,0.500000,0.000000,4.674511,-2.591338,0.00,0.000000,0.0,0.00,0.0,16.000000,0,0,1.5,1
1,DE-BB,2020-02-23,0.142857,0.000000,1.142857,-1.285714,-0.285714,0.857143,1.519186,-1.302753,0.00,0.000000,0.0,0.00,0.0,16.000000,0,0,6.0,2
2,DE-BB,2020-03-01,3.285714,7.571429,12.428571,0.142857,0.285714,0.714286,-3.389097,-1.281404,0.00,0.000000,0.0,0.00,0.0,51.857143,0,0,13.0,3
3,DE-BB,2020-03-08,5.428571,7.285714,16.000000,-1.142857,0.857143,0.857143,-8.297381,-5.640300,0.00,0.000000,0.0,0.00,0.0,515.428571,0,0,20.0,4
4,DE-BB,2020-03-15,-2.000000,8.000000,26.428571,-7.142857,-1.000000,2.000000,-13.205664,-12.539735,0.00,0.000000,0.0,0.00,0.0,2953.428571,0,0,27.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46746,SE-Z,2022-09-18,-13.571429,0.285714,0.000000,-38.714286,-12.857143,1.571429,-19.039363,-15.274852,11.11,16.670000,0.0,19.05,0.0,0.000000,1,1,944.0,136
46747,SE-Z,2022-09-25,-9.571429,3.000000,0.000000,-37.571429,-11.428571,0.857143,-19.144914,-12.287451,11.11,14.878571,0.0,17.01,0.0,0.000000,1,1,951.0,137
46748,SE-Z,2022-10-02,-8.571429,-0.142857,0.000000,-40.142857,-10.142857,0.714286,-19.250465,-10.791190,11.11,14.580000,0.0,16.67,0.0,0.000000,1,1,958.0,138
46749,SE-Z,2022-10-09,-15.571429,-3.714286,0.000000,-42.714286,-10.285714,1.571429,-19.356016,-12.093726,11.11,14.580000,0.0,16.67,0.0,0.000000,1,1,965.0,139


In [346]:
datos_mob = datos_mob.rename(columns={'iso_3166_2_code': 'iso_code'})

In [347]:
datos_mob['date'] = pd.to_datetime(datos_mob['date'])


In [348]:
datos_mob['year'] = pd.to_datetime(datos_mob['date']).dt.year

In [349]:
dataset_weekly= datos_mob.merge(df, left_on=['year', 'iso_code'], right_on=['year', 'iso_code'], how='left').fillna(method='ffill')


C:\Users\34645\AppData\Local\Temp\ipykernel_14300\2842115797.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  dataset_weekly= datos_mob.merge(df, left_on=['year', 'iso_code'], right_on=['year', 'iso_code'], how='left').fillna(method='ffill')


In [350]:
dataset_weekly

,iso_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_raw,residential_percent_change_from_baseline,trend,workplaces,...,edu2,edu3,age_dependency,old_age_dependency,young_age_dependency,sex_ratio,unemp,f_unemp,m_unemp,foreigners
0,DE-BB,2020-02-16,6.500000,2.500000,17.000000,4.500000,0.500000,0.000000,4.674511,-2.591338,...,NaN,NaN,0.704082,0.282054,0.429700,105.526887,6.0,0.073708,0.044889,4.0
1,DE-BB,2020-02-16,6.500000,2.500000,17.000000,4.500000,0.500000,0.000000,4.674511,-2.591338,...,NaN,NaN,0.631870,0.320856,0.319111,106.904702,4.0,0.049003,0.027374,4.0
2,DE-BB,2020-02-16,6.500000,2.500000,17.000000,4.500000,0.500000,0.000000,4.674511,-2.591338,...,NaN,NaN,0.743959,0.280332,0.476880,102.895129,7.0,0.082430,0.046882,3.0
3,DE-BB,2020-02-16,6.500000,2.500000,17.000000,4.500000,0.500000,0.000000,4.674511,-2.591338,...,NaN,NaN,0.687392,0.279542,0.417556,104.191249,5.0,0.067109,0.041551,3.0
4,DE-BB,2020-02-23,0.142857,0.000000,1.142857,-1.285714,-0.285714,0.857143,1.519186,-1.302753,...,NaN,NaN,0.704082,0.282054,0.429700,105.526887,6.0,0.073708,0.044889,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80206,SE-Z,2022-09-18,-13.571429,0.285714,0.000000,-38.714286,-12.857143,1.571429,-19.039363,-15.274852,...,0.567431,0.468095,0.750466,0.408183,0.342284,99.646277,9.0,0.097663,0.087054,6.0
80207,SE-Z,2022-09-25,-9.571429,3.000000,0.000000,-37.571429,-11.428571,0.857143,-19.144914,-12.287451,...,0.567431,0.468095,0.750466,0.408183,0.342284,99.646277,9.0,0.097663,0.087054,6.0
80208,SE-Z,2022-10-02,-8.571429,-0.142857,0.000000,-40.142857,-10.142857,0.714286,-19.250465,-10.791190,...,0.567431,0.468095,0.750466,0.408183,0.342284,99.646277,9.0,0.097663,0.087054,6.0
80209,SE-Z,2022-10-09,-15.571429,-3.714286,0.000000,-42.714286,-10.285714,1.571429,-19.356016,-12.093726,...,0.567431,0.468095,0.750466,0.408183,0.342284,99.646277,9.0,0.097663,0.087054,6.0


In [352]:
ruta_archivo_csv = r'C:\Users\34645\Desktop\Doctorado_II\Códigos_Python\IEDIS\Data\eu_weeklydataset.csv'
dataset_weekly.to_csv(ruta_archivo_csv, index=False)  # index=False para no incluir el índice en el archivo CSV